In [8]:
import warnings
from typing import Iterable

import pandas as pd
import datasets
import sympy
import lark

import gadgets

In [9]:
df = {}
for split in ["train", "test", "val"]:
    print(split)
    df[split] = pd.read_csv(f"../data/mawps/original/{split}.csv", names=["problem", "equation", "problem_arabic"])[["problem", "problem_arabic", "equation"]]
    df[split]["expression"] = df[split]["equation"].str.replace(" ", "").str.removeprefix("x=").str.removesuffix("=x")
    df[split]["equation"] = df[split]["equation"]
    df[split]["question"] = df[split]["problem"].str.strip()
    #df[split]["question_arabic"] = df[split]["problem_arabic"].str.strip()
    del df[split]["problem"]
    del df[split]["problem_arabic"]
    print(df[split][df[split]["expression"].str.contains("x")].index)

train
Index([ 123,  211,  384,  746,  757,  855, 1241, 1837, 1842, 1949, 2033, 2222,
       2369, 2452, 2493, 2564, 2699, 3143, 3352, 3563, 3597, 3623],
      dtype='int64')
test
Index([2, 175, 253, 388], dtype='int64')
val
Index([70, 108, 618, 728, 777, 778, 896], dtype='int64')


In [10]:
df["train"][df["train"]["expression"].str.contains("x")]

,equation,expression,question
123,85.0=(15.0*x)-20.0,85.0=(15.0*x)-20.0,You make candles and sell them for 15 dollars ...
211,35+x=83,35+x=83,Arthur baked 35 muffins. How many more muffins...
384,17+(7*x)=80,17+(7*x)=80,Oceanside Bike Rental Shop charges 17 dollars ...
746,x*(1.0-(8.0*0.01))=13580.0,x*(1.0-(8.0*0.01))=13580.0,"A disc jockey now charges 13,580 dollars for a..."
757,(x-6.0)*4.0=36.0,(x-6.0)*4.0=36.0,If I subtract 6 from a number and then multipl...
855,1.3333*x=4.82,1.3333*x=4.82,"1.3333 of a number is 4.82 , what is the number?"
1241,22+x=55,22+x=55,There are 22 walnut trees currently in the par...
1837,(0.2*x)-4.0=6.0,(0.2*x)-4.0=6.0,0.20 of a number decreased by 4 is equal to 6....
1842,((450.0+200.0)+x)/3.0=300.0,((450.0+200.0)+x)/3.0=300.0,Martha wants to buy a digital camera from her ...
1949,53=7+14+x,53=7+14+x,Randy needs 53 cupcakes for a birthday party. ...


In [11]:
df["train"].loc[123, "expression"] = "(85.0+20)/15.0"
df["train"].loc[211, "expression"] = "83-35"
df["train"].loc[384, "expression"] = "(80-17)/7"
df["train"].loc[746, "expression"] = "13580.0/(1.0-(8.0*0.01))"
df["train"].loc[757, "expression"] = "(36.0/4.0)+6"
df["train"].loc[855, "expression"] = "4.82/1.3333"
df["train"].loc[1241, "expression"] = "55-22"
df["train"].loc[1837, "expression"] = "(6.0+4.0)/0.2"
df["train"].loc[1842, "expression"] = "(300.0*3.0)-(450.0+200.0)"
df["train"].loc[1949, "expression"] = "53-(7+14)"
df["train"].loc[2033, "expression"] = "((-12.0) + (-8.0)) / 2.0"
df["train"].loc[2222, "expression"] = "(2.0+1.0)/0.6"
df["train"].loc[2369, "expression"] = "(12-8)*2"
df["train"].loc[2452, "expression"] = "(13.0+7.0)/4.0"
df["train"].loc[2493, "expression"] = "7-3"
df["train"].loc[2564, "expression"] = "1110/30"
df["train"].loc[2699, "expression"] = "(30.0-26.48)/(16*.01)"
df["train"].loc[3143, "expression"] = "9792-3513"
df["train"].loc[3352, "expression"] = "56-35"
df["train"].loc[3563, "expression"] = "1.89+0.28"
df["train"].loc[3597, "expression"] = "(70.0-35.68)/8.58"
df["train"].loc[3623, "expression"] = "20-(2*5.92)-6.79"

In [12]:
df["val"][df["val"]["expression"].str.contains("x")]

,equation,expression,question
70,(5.0*24.95)+(0.49*x)=200.0,(5.0*24.95)+(0.49*x)=200.0,A car rental agency advertised renting a luxur...
108,5*x=50,5*x=50,It has been tradition in Shiela’s home to hang...
618,1450.0/1127.0=3.0/x,1450.0/1127.0=3.0/x,The Sears Tower in Chicago is 1450 feet tall. ...
728,190.0+((4.0*0.01)*x)=500.0,190.0+((4.0*0.01)*x)=500.0,Trenton sells electronic supplies. Each week h...
777,8.77+10.97+x=20,8.77+10.97+x=20,"Joan paid $8.77 on a cat toy, and a cage cost ..."
778,20.99+((25.0*0.01)*x)=95.74,20.99+((25.0*0.01)*x)=95.74,Michael rented a truck for one day. There was ...
896,(10.0*4.2)+(90.0*5.6)=(10.0+90.0)*x,(10.0*4.2)+(90.0*5.6)=(10.0+90.0)*x,Find the cost per pound of a trail mix made fr...


In [13]:
df["val"].loc[70, "expression"] = "(200.0-(5.0*24.95))/0.49"
df["val"].loc[108, "expression"] = "50/5"
df["val"].loc[618, "expression"] = "3.0/(1450.0/1127.0)"
df["val"].loc[728, "expression"] = "(500.0-190.0)/(4.0*0.01)"
df["val"].loc[777, "expression"] = "20-8.77-10.97"
df["val"].loc[778, "expression"] = "(95.74-20.99)/(25.0*0.01)"
df["val"].loc[896, "expression"] = "((10.0*4.2)+(90.0*5.6))/(10.0+90.0)"

In [14]:
df["test"][df["test"]["expression"].str.contains("x")]

,equation,expression,question
2,56-x=22,56-x=22,"Dan found 56 seashells on the beach, he gave J..."
175,73+x=96,73+x=96,There were 73 bales of hay in the barn. Jason ...
253,5.0+x=20.0,5.0+x=20.0,The sum of 5 and a number is 20. Find the number.
388,34+x=86,34+x=86,Mike had 34 peaches left at his roadside fruit...


In [15]:
df["test"].loc[2, "expression"] = "56-22"
df["test"].loc[175, "expression"] = "96-73"
df["test"].loc[253, "expression"] = "20-5.0"
df["test"].loc[388, "expression"] = "86-34"

In [16]:
def solve_str_eq(string):
    lhs, rhs = string.split("=")
    lhs = sympy.sympify(lhs)
    rhs = sympy.sympify(rhs)
    return sympy.solve(lhs - rhs)[0]

In [17]:
for name, split in df.items():
    expr_result = split["expression"].apply(sympy.parse_expr)
    eq_solution = split["equation"].apply(solve_str_eq)
    print(((expr_result - eq_solution).apply(abs) < 0.000001).mean())

1.0
1.0
1.0


In [18]:
grammar = """
?start: expr

?expr: neg

?atom: num
    | implicit_mul
    | "(" expr ")"

implicit_mul: num ( "(" expr ")" )+
            | "(" expr ")" ( "(" expr ")" )+

?neg: add
    | "-" neg -> neg
    | "-" add -> neg
?add: sub
    | sub ("+" sub)+ -> add
?sub: mul
    | mul ("-" mul)+ -> sub
?mul: div
    | div ("*" div)+ -> mul
?div: pow
    | pow ("/" pow)+ -> div
?pow: perc
    | perc ("**" perc)+ -> pow
?perc: atom "%" -> perc
     | atom
?num: SIGNED_NUMBER

%import common.SIGNED_NUMBER
%import common.WS
%ignore WS
"""

In [19]:
class TreeEvaluator:

    def __init__(self, calc: gadgets.gadget.Calculator, parser: lark.Lark) -> None:
        self.cache = {}
        self.calc = calc
        self.parser = parser

    def eval_tree(self, tree: lark.Tree | lark.Token) -> tuple[str, sympy.Expr]:
        if tree not in self.cache:
            self.cache[tree] = self._eval_tree(tree)
        return self.cache[tree]

    def _eval_tree(self, tree: lark.Tree | lark.Token) -> tuple[str, sympy.Expr]:
        if isinstance(tree, lark.Token):
            if tree.type in ("SIGNED_NUMBER", "NUMBER"):
                return None, self.calc.evaluate(tree.value)
            else:
                raise ValueError(f"unknown token {tree}")
        
        assert isinstance(tree.data, str)
        operation = tree.data
        args_nodes = tree.children
        args = [self._format_arg(self.eval_tree(arg_node)[1]) for arg_node in args_nodes]
        inputs = self._format_op(operation, args)
        return inputs, self.calc.evaluate(inputs)
    
    def _format_op(self, op: str, args: list[str]) -> str:
        if op == "neg":
            assert len(args) == 1
            return "-" + args[0]
        if op == "add" or op == "implicit_add":
            return " + ".join(args)  
        if op == "sub":
            return " - ".join(args)
        if op == "mul" or op == "implicit_mul":
            return " * ".join(args)
        if op == "div":
            return " / ".join(args)
        if op == "pow":
            return " ** ".join(args)
        if op == "perc":
            assert len(args) == 1
            return f"{args[0]} / 100"
        raise ValueError(f"unknown operation {op}")

    def _format_arg(self, value_expr: sympy.Number) -> str:
        value_str = self.calc.format_sympy_number(value_expr, add_approx=False)
        if isinstance(value_expr, sympy.core.function.Application):
            return value_str
        if isinstance(value_expr, (sympy.Float, sympy.Integer, sympy.NumberSymbol)):
            if value_expr < 0:
                return "(" + value_str + ")"
            return value_str
        if isinstance(value_expr, sympy.Rational):
            return "(" + value_str + ")"
        if isinstance(value_expr, (sympy.Mul, sympy.Pow, sympy.Add)):
            return "(" + value_str + ")"
        warnings.warn(f"weird value type {type(value_expr)} for {value_expr} (string: '{value_str}')")
        return "(" + value_str + ")"

    def dfs(self, tree: lark.Tree | lark.Token) -> Iterable[lark.Tree | lark.Token]:
        if isinstance(tree, lark.Tree):
            for child in tree.children:
                yield from self.dfs(child)
        yield tree

    def expr_to_steps(self, expr: str, drop_repeated: bool = True) -> tuple[list[gadgets.datatypes.Interaction], sympy.Expr]:
        tree = self.parser.parse(expr)
        steps = []
        for subtree in self.dfs(tree):
            step = self.eval_tree(subtree)
            if step is None:
                continue
            if step[0] is None:
                continue
            inputs, output_expr = step
            interaction = gadgets.datatypes.Interaction(
                gadget_id="calculator",
                inputs=inputs,
                outputs=self.calc.format_sympy_number(output_expr),
            )
            if drop_repeated and interaction in steps:
                continue
            steps.append(interaction)
            
        _, result = self.eval_tree(tree)
        return steps, result
    

In [20]:
df["train"]["expression"].sample(10, random_state=0)

3062               107-20
566                59+101
981                 82+59
1009    ((45.0-35.0)/5.0)
2470               155-42
2921                18+70
3379                 54/7
501                382/10
1292                76-23
1403                 37-3
Name: expression, dtype: object

In [21]:
calc = gadgets.gadget.Calculator()

parser = lark.Lark(grammar)
tree_evaluator = TreeEvaluator(calc, parser)

for split in ["train", "test", "val"]:
    df[split]["chain"], df[split]["result"] = zip(*df[split]["expression"].apply(tree_evaluator.expr_to_steps))
    df[split]["result_float"] = df[split]["result"].apply(float)
    df[split]["result"] = df[split]["result"].apply(calc.format_sympy_number, add_approx=False)
    df[split]["chain"] = df[split].apply(lambda row: gadgets.markup.to_model_markup(chain=row["chain"], result=row["result"]), axis=1).apply(str).str.strip()

In [22]:
df["train"].sample(10, random_state=0)[["expression", "chain", "result", "result_float"]]

,expression,chain,result,result_float
3062,107-20,"<gadget id=""calculator"">107 - 20</gadget>\n<ou...",87,87.000000
566,59+101,"<gadget id=""calculator"">59 + 101</gadget>\n<ou...",160,160.000000
981,82+59,"<gadget id=""calculator"">82 + 59</gadget>\n<out...",141,141.000000
1009,((45.0-35.0)/5.0),"<gadget id=""calculator"">45 - 35</gadget>\n<out...",2,2.000000
2470,155-42,"<gadget id=""calculator"">155 - 42</gadget>\n<ou...",113,113.000000
2921,18+70,"<gadget id=""calculator"">18 + 70</gadget>\n<out...",88,88.000000
3379,54/7,"<gadget id=""calculator"">54 / 7</gadget>\n<outp...",54/7,7.714286
501,382/10,"<gadget id=""calculator"">382 / 10</gadget>\n<ou...",191/5,38.200000
1292,76-23,"<gadget id=""calculator"">76 - 23</gadget>\n<out...",53,53.000000
1403,37-3,"<gadget id=""calculator"">37 - 3</gadget>\n<outp...",34,34.000000


In [23]:
for key, value in df["train"].loc[1009].to_dict().items():
    print(key)
    print(value)
    print()

equation
x=((45.0-35.0)/5.0)

expression
((45.0-35.0)/5.0)

question
Wendy picked 45 flowers for her friend’s wedding. She was making bouquets with 5 flowers in each one. If 35 of the flowers wilted before the wedding, how many bouquets could she still make?

chain
<gadget id="calculator">45 - 35</gadget>
<output>10</output>

<gadget id="calculator">10 / 5</gadget>
<output>2</output>

<result>2</result>

result
2

result_float
2.0



In [24]:
import string
import random

id_chars = list(string.ascii_letters + string.digits)

ds = datasets.DatasetDict()

for name, split in df.items():
    random.seed(f"mawps/{name}")
    split["id"] = "mawps__" + pd.Series(["".join(random.choices(id_chars, k=16)) for _ in range(len(split))])
    split_reorder = split[["id", "question", "chain", "result", "result_float", "equation", "expression"]]
    ds[name] = datasets.Dataset.from_pandas(split_reorder)
    split_reorder.to_json(f"../data/mawps/processed/{name}.jsonl", orient="records", lines=True, force_ascii=False)


In [27]:
pd.concat(df.values())["id"].is_unique

True

In [28]:
# ds.push_to_hub("MU-NLPC/Calc-mawps", "original-splits")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
datasets.load_dataset("MU-NLPC/Calc-mawps", "original-splits")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/3636 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/1040 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'question', 'chain', 'result', 'result_float', 'equation', 'expression'],
        num_rows: 520
    })
    train: Dataset({
        features: ['id', 'question', 'chain', 'result', 'result_float', 'equation', 'expression'],
        num_rows: 3636
    })
    val: Dataset({
        features: ['id', 'question', 'chain', 'result', 'result_float', 'equation', 'expression'],
        num_rows: 1040
    })
})

In [30]:
datasets.load_dataset("MU-NLPC/Calc-mawps", "original-splits")["train"][0]

{'id': 'mawps__ANMCjEDDzkz5L66p',
 'question': 'A painter needed to paint 12 rooms in a building. Each room takes 7 hours to paint. If he already painted 5 rooms, how much longer will he take to paint the rest?',
 'chain': '<gadget id="calculator">12 - 5</gadget>\n<output>7</output>\n\n<gadget id="calculator">7 * 7</gadget>\n<output>49</output>\n\n<result>49</result>',
 'result': '49',
 'result_float': 49.0,
 'equation': 'x=(7.0*(12.0-5.0))',
 'expression': '(7.0*(12.0-5.0))'}